In [1]:
import torch
import sys
# from InvertedResidual_small_Unet import invertedResidual_unet
sys.path.append('../Model')
from InvertedResidual_large_Unet import invertedResidual_unet
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
import pandas as pd
import sys
import argparse
import h5py
from scipy.io import savemat
from tqdm import tqdm
from tqdm.notebook import tqdm

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class MyDataset(Dataset):

    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, index):
        return torch.Tensor(np.array(self.x_data[index])), torch.Tensor(
            np.array(self.y_data[index]))

def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder created: {folder_path}")
    else:
        print(f"Folder already exists: {folder_path}")

In [ ]:
import os
import glob
tol_li = []
for i in range(1,11):
    directory = 'weight_model_4_utrans_unet_InSmUNet/InvertedResidual_UNet_large/'
    pattern = 'InvertedResidual_UNet_largeFold%02d*' % i  
    file_list = glob.glob(os.path.join(directory, pattern))
    if not file_list:
        print("no\n")
    file_list.sort()
    last_file = file_list[-1]
    tol_li.append(last_file)
tol_li


In [5]:
BATCH_SIZE = 1024
tol_abp_true = []
tol_abp_pred = []
weight_ten = tol_li


In [ ]:
for fold_number in range(1,11):
    print("fold_number = ", fold_number)
    dataset_dir_path = '/home/yonghu/data_yonghu/code_python/VitalDB_DataLoader_tol/Folder%02d/' % fold_number
    weight_path = weight_ten[fold_number - 1]
    test_data_set = torch.load(dataset_dir_path+"test_dataset.pt")
    print(len(test_data_set))
    test_loader = DataLoader(test_data_set, batch_size=BATCH_SIZE, shuffle=False)
    print(len(test_loader))
    
    model = invertedResidual_unet().to(device)
    checkpoint = torch.load(weight_path)

    model_state_dict = checkpoint['model'] if 'model' in checkpoint else checkpoint
    # model_list[1].load_state_dict(torch.load(Teacher_Path))
    model.load_state_dict(model_state_dict, strict=False)
    
    model.eval()
    true_data = []
    pred_data = []
    with torch.no_grad():
        for inputs, targets in tqdm(test_loader):
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            true_data.append(targets.detach().cpu().numpy().flatten())
            pred_data.append(outputs.detach().cpu().numpy().flatten())
    true_data = np.concatenate(true_data)
    pred_data = np.concatenate(pred_data)
    tol_abp_true.append(true_data)
    tol_abp_pred.append(pred_data)

    

In [ ]:
tol_abp_true_np = np.concatenate(tol_abp_true)
tol_abp_pred_np = np.concatenate(tol_abp_pred)
print(tol_abp_true_np.shape)
print(tol_abp_pred_np.shape)

In [ ]:
mae = np.mean(np.abs(tol_abp_pred_np - tol_abp_true_np))
errors = tol_abp_pred_np - tol_abp_true_np
std = np.std(errors)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Standard Deviation (STD): {std}")